In [ ]:
#CELL 1
#@title Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }
#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>


In [1]:
#@title ##**Start TextGenWebUI**
#GOOGLE COLAB ONLY
import os, signal
from IPython.utils import capture

# Find any existing cloudflared processes
result = os.popen('ps -A | grep cloudflared').read()
if 'cloudflared' in result:
    # If a cloudflared process is found, kill it
    os.kill(int(result.split()[0]), signal.SIGKILL)

#@markdown ####**Configure and Run**
text_streaming = False #@param {type:"boolean"}
GetAPI = True #@param {type:"boolean"}

#Parameters
huggingface_org="TheBloke" #@param{type:"string"}
huggingface_repo="Echidna-13B-v0.3-GPTQ" #@param{type:"string"}
model = huggingface_repo
huggingface_branch="main" #@param{type:"string"}

##### Oobabooga ######
#Main Directories:
main_dir = "/content/text-generation-webui"

def install_ooba(): #$
  %cd /content
  !mkdir text-generation-webui/logs
  !ln -s text-generation-webui/logs .
  !ln -s text-generation-webui/characters .
  !ln -s text-generation-webui/models .
  !rm -r sample_data

  !wget https://oobabooga.github.io/settings-colab.json -O settings-colab-template.json

  %cd text-generation-webui
  !apt-get -y install -qq aria2
  !pip install -r requirements.txt flask_cloudflared

def download_model(huggingface_repobranch, huggingface_org, huggingface_repo, huggingface_branch): #$
    model_loc = f"/content/models/{huggingface_repo}"
    tmp_repo = f"/content/.{huggingface_repo}"
    !git lfs install --skip-smudge && export GIT_LFS_SKIP_SMUDGE=1 && git clone https://huggingface.co/{huggingface_org}/{huggingface_repo} {tmp_repo} --branch {huggingface_branch}
    !rm -rf {tmp_repo}/pytorch_model.bin.index.json {tmp_repo}/training_args.bin {tmp_repo}/trainer_state.json {tmp_repo}/all_results.json {tmp_repo}/eval_results.json {tmp_repo}/.gitattributes {tmp_repo}/train_results.json
    repo = os.listdir(tmp_repo);
    for download in repo:
      link_path = f"https://huggingface.co/{huggingface_org}/{huggingface_repo}/"
      if ".json" in download or ".txt" in download:
        link_path += f"raw/{huggingface_branch}/{download}"
      else: link_path += f"resolve/{huggingface_branch}/{download}"
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {link_path} -d {model_loc} -o {download}

print("\033[92m[Installing oobabooga]")
if (not os.path.exists(main_dir) or ('GPTQ' in model or '4bit' in model or '128' in model) and not os.path.exists(f"{main_dir}/repositories")):
  %cd /content
  !git clone https://github.com/oobabooga/text-generation-webui/
  %cd /content/text-generation-webui
  !git config advice.detachedHead false
  !git checkout 019371c
  %cd /content

with capture.capture_output() as cap:
    install_ooba();
%cd /content

huggingface_repobranch=""
if not "main" == huggingface_branch: huggingface_repobranch=f"{huggingface_repo}_{huggingface_branch}"
else:huggingface_repobranch=huggingface_repo

print(f"\033[92m[Installing Model {huggingface_repo}]")
if not os.path.exists(f"/content/models/{huggingface_repo}"):
    download_model(huggingface_repobranch, huggingface_org, huggingface_repo, huggingface_branch);

#Server starting
print("\033[92m[Starting Server]")
params = set()
if ('4bit' in model): params.add('--wbits 4 --model_type Llama')
if ('128' in model): params.add('--groupsize 128')

if GetAPI: params.add('--public-api')
if ('gptq' in model.lower()):
    params.add("--loader exllama_hf --max_seq_len 7500 --alpha_value 2.565")
else:
    params.add("--loader llamacpp")

active_extensions = []
if GetAPI: active_extensions.append('api')
if len(active_extensions) > 0: params.add(f'--{"".join(active_extensions)}')
print("\033[1m "+"\033[92m ")

def run_ooba():#$
  %cd {main_dir}
  cmd = f"python server.py --verbose --model {huggingface_repobranch} {' '.join(params)}"
  print(cmd)
  !$cmd

if GetAPI:
    print("\033[92m[Running Ooba...]")
    run_ooba()

[Installing oobabooga]
/content
Cloning into 'text-generation-webui'...
remote: Enumerating objects: 14492, done.
remote: Counting objects: 100% (1943/1943), done.
remote: Compressing objects: 100% (350/350), done.
remote: Total 14492 (delta 1702), reused 1749 (delta 1591), pack-reused 12549
Receiving objects: 100% (14492/14492), 25.77 MiB | 25.30 MiB/s, done.
Resolving deltas: 100% (10028/10028), done.
/content/text-generation-webui
HEAD is now at 019371c Lint
/content
/content
[Installing Model Echidna-13B-v0.3-GPTQ]
Git LFS initialized.
Cloning into '/content/.Echidna-13B-v0.3-GPTQ'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 56 (delta 27), reused 0 (delta 0), pack-reused 3
Unpacking objects: 100% (56/56), 495.04 KiB | 2.95 MiB/s, done.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
e57e7c|OK  